In [3]:
import os
import requests
from dotenv import load_dotenv


# Carregar as variáveis de ambiente'
load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")


In [4]:

def get_optimized_route(start_location, stop_locations):
    # Endpoint da API de Rotas
    directions_url = "https://maps.googleapis.com/maps/api/directions/json"

    # Definir parâmetros da solicitação
    params = {
        "origin": start_location,
        "destination": start_location, 
        "waypoints": "|".join(stop_locations),
        "optimize_waypoint": "true",
        "key": GOOGLE_MAPS_API_KEY
    }

    # Realizar a solicitação
    response = requests.get(directions_url, params=params)
    response_data = response.json()

    # Verificar erros
    if response.status_code != 200 or response_data.get("status") != "OK":
        print("Erro ao obter rota otimizada:", response_data.get("error_message"))
        return None

    # Extrair informações relevantes
    route_info = {
        "total_distance": 0,
        "total_duration": 0,
        "optimized_order": [],
        "steps": []
    }

    # Analisar os dados da resposta
    route = response_data["routes"][0]
    legs = route["legs"]
    for idx, leg in enumerate(legs):
        route_info["total_distance"] += leg["distance"]["value"]
        route_info["total_duration"] += leg["duration"]["value"]
        route_info["steps"].append({
            "start_address": leg["start_address"],
            "end_address": leg["end_address"],
            "distance": leg["distance"]["text"],
            "duration": leg["duration"]["text"]
        })

    # Ordem otimizada das paradas
    route_info["optimized_order"] = route["waypoint_order"]

    # Converter distância total e duração para formatos legíveis
    route_info["total_distance"] = f"{route_info['total_distance'] / 1000:.2f} km"
    route_info["total_duration"] = f"{route_info['total_duration'] / 60:.2f} mins"

    print(response_data)
    return route_info


start = "Av. Unisinos, 950 - Cristo Rei, São Leopoldo - RS, 93022-750"
stops = [
    "R. Primeiro de Março, 821 - Centro, São Leopoldo - RS, 93010-210",
    "Cemitério Municipal - Cristo Rei, São Leopoldo - RS, 93020-190",
    "R. Pres. João Goulart, 1080 - Morro do Espelho, São Leopoldo - RS, 93030-090",
]
route_data = get_optimized_route(start, stops)

if route_data:
    print("Distância total:", route_data["total_distance"])
    print("Duração total:", route_data["total_duration"])
    print("Ordem otimizada das paradas:", route_data["optimized_order"])
    print("Detalhes das paradas:")
    for step in route_data["steps"]:
        print(f"{step['distance']}, {step['duration']}")


{'geocoded_waypoints': [{'geocoder_status': 'OK', 'place_id': 'ChIJtQDan0RpGZUROspmDfV3axw', 'types': ['street_address']}, {'geocoder_status': 'OK', 'place_id': 'ChIJ1xsBRFBoGZURBbyytSc733c', 'types': ['premise']}, {'geocoder_status': 'OK', 'place_id': 'ChIJVVVV-floGZURCqRn1jjn0S4', 'types': ['cemetery', 'establishment', 'point_of_interest']}, {'geocoder_status': 'OK', 'place_id': 'ChIJVzFmDgJpGZURhil2nep4dXk', 'types': ['premise']}, {'geocoder_status': 'OK', 'place_id': 'ChIJtQDan0RpGZUROspmDfV3axw', 'types': ['street_address']}], 'routes': [{'bounds': {'northeast': {'lat': -29.768552, 'lng': -51.1371188}, 'southwest': {'lat': -29.7986827, 'lng': -51.1629138}}, 'copyrights': 'Map data ©2024', 'legs': [{'distance': {'text': '4.9 km', 'value': 4891}, 'duration': {'text': '10 mins', 'value': 574}, 'end_address': 'R. Primeiro de Março, 821 - Centro, São Leopoldo - RS, 93010-210, Brazil', 'end_location': {'lat': -29.769201, 'lng': -51.1466214}, 'start_address': 'Av. Unisinos, 950 - Cristo 